In [7]:
import pyaudio
import wave
import scipy
from scipy.io import wavfile
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import cm
from pylab import rcParams
import librosa
import librosa.display
import pandas as pd

import cv2

import IPython
import numpy as np
import sys
import os
import pandas as pd

from tqdm import tqdm
from shutil import copyfile
from keras import optimizers, regularizers
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.callbacks import  History, ReduceLROnPlateau, CSVLogger
from keras.layers import Dense, Activation, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from keras.layers import Dropout
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model


from sklearn.model_selection import train_test_split  
from sklearn import svm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import pywt

import seaborn as sns  
import tensorflow as tf
tf.test.is_gpu_available()
%matplotlib_inline

UsageError: Line magic function `%matplotlib_inline` not found.


In [2]:
dirname = 'Processed_Dataset'

In [3]:
#Total number of files
Total_files = 0
Files = []
Files_loc = []
for i in range(len(os.listdir(os.path.join(os.getcwd(), 'Processed_Dataset')))):
    folder_name = os.listdir(os.path.join(os.getcwd(), 'Processed_Dataset'))[i]
    files = os.listdir(os.path.join(os.getcwd(), 'Processed_Dataset', folder_name))
    Fil = [os.path.join(os.getcwd(), 'Processed_Dataset', folder_name,  f) for f in files]  # Absolute location of files
    Files_loc.append(Fil)
#     files = ['Female_' + x for x in files if i%2]
#     files = ['Male_' + x for x in files if not i%2]
    
    Files.append(files)

training_files = []
training_files_loc = []
for i, x in enumerate(Files):
    
    # Omitting Calm and Neutral emotions from the training database
    for file in Files_loc[i]:
        if file.split('\\')[7].split('-')[0] == 'Calm' or file.split('\\')[7].split('-')[0] == 'Neutral':
            continue
        else:
            training_files_loc.append(file)
    if i%2:
        for file in x:
            training_files.append('Female_' + file)
    else:
        for file in x:
            training_files.append('Male_' + file)

In [4]:
print(len(Files))
print(len(Files_loc[23]))

24
60


In [5]:
print(training_files_loc[48])
print(training_files[60])
print(Files[0][0][:-4])

D:\PDF\7th_Sem\BTP\Model\Processed_Dataset\Actor_02\Angry-Normal-01-01.wav
Female_Angry-Normal-01-01.wav
Angry-Normal-01-01


In [20]:
spec_dir = 'Spectrograms'
rcParams['figure.figsize'] = 5, 5
c = 0
for type_of_spec in ['linear', 'log']:
    if not os.path.exists(os.path.join(os.getcwd(), spec_dir, type_of_spec)):
        os.mkdir(os.path.join(os.getcwd(), spec_dir, type_of_spec))
        
for i in range(len(os.listdir(os.path.join(os.getcwd(), dirname)))):
    folder_name = os.listdir(os.path.join(os.getcwd(), dirname))[i]   # select i'th folder in the directory
    
    for type_of_spec in ['linear', 'log']:
        if not os.path.exists(os.path.join(os.getcwd(), spec_dir, type_of_spec, folder_name)):
            os.mkdir(os.path.join(os.getcwd(), spec_dir, type_of_spec, folder_name))
        
        for file in Files_loc[i]:
            if file.split('\\')[7].split('-')[0] == 'Calm' or file.split('\\')[7].split('-')[0] == 'Neutral':
                continue
            else:
                audio_path = file
                samplingFrequency, signalData = wavfile.read(audio_path)
                stft = np.abs(librosa.stft(signalData, win_length=512, hop_length=128))
                
                
                fig,ax = plt.subplots(1)

                plt.ioff()
                fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
                ax.axis('off')
                # D = librosa.amplitude_to_db(stft, ref=np.max)
                librosa.display.specshow(librosa.amplitude_to_db(stft, ref=np.max), 
                                         hop_length=128, 
                                         x_axis='time', 
                                         y_axis=type_of_spec, 
                                         cmap=cm.jet)

                fig.savefig(os.path.join(os.getcwd(), spec_dir, type_of_spec, folder_name, file.split('\\')[7])[:-4], 
                            dpi=64, frameon='False')
                plt.close()

In [18]:
spec_dir = 'Spectrograms'
audio_path = Files_loc[0][5]
samplingFrequency, signalData = wavfile.read(audio_path)
stft = np.abs(librosa.stft(signalData, win_length=512, hop_length=128))


rcParams['figure.figsize'] = 5, 5
fig,ax = plt.subplots(1)

plt.ioff()
fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
ax.axis('off')
# D = librosa.amplitude_to_db(stft, ref=np.max)
librosa.display.specshow(librosa.amplitude_to_db(stft, ref=np.max), 
                         hop_length=128, 
                         x_axis='time', 
                         y_axis='linear', 
                         cmap=cm.jet)
# plt.title('Power spectrogram')
# plt.colorbar(format='%+2.0f dB')
# plt.tight_layout()

fig.savefig(os.path.join(os.getcwd(), spec_dir, 'log', 'First'), dpi=64, frameon='False')


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\PDF\\7th_Sem\\BTP\\Model\\Spectrograms\\log\\First.png'

In [17]:

# print(im.shape)
# plt.imshow(im)
# plt.imshow(im[:,:,:3])


img = mpimg.imread(os.path.join(os.getcwd(), spec_dir, 'log', 'First.png'))
print(img.shape)
mpimg.imsave(os.path.join(os.getcwd(), spec_dir, 'log', 'out.png'), img[:,:,:3])
plt.imshow(img)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\PDF\\7th_Sem\\BTP\\Model\\Spectrograms\\log\\First.png'

In [10]:
Files_loc[0][0].split('\\')[7]

'Angry-Normal-01-01.wav'